In [4]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.1",
    temperature=0
)
llm

ChatOllama(model='llama3.1', temperature=0.0)

###Invocation

In [9]:
from langchain_core.messages import AIMessage

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Chinese, Transalte the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="Here's the translation:\n\n我喜欢编程。 (Wǒ xǐhuān biānxí.)", response_metadata={'model': 'llama3.1', 'created_at': '2024-08-16T18:17:38.843924Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 16225877000, 'load_duration': 8415475300, 'prompt_eval_count': 37, 'prompt_eval_duration': 3489386000, 'eval_count': 24, 'eval_duration': 4282241000}, id='run-f380955e-a058-4229-a6bf-8fb6f960e61f-0', usage_metadata={'input_tokens': 37, 'output_tokens': 24, 'total_tokens': 61})

In [10]:
print(ai_msg.content)

Here's the translation:

我喜欢编程。 (Wǒ xǐhuān biānxí.)


### Chaining
We can chain our model with a prompt template like so:

In [13]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        (
            "system",
            "You are a helpful assistant that translate {input_language} to {output_language}.",
        ),
        ("human", "{input}",)
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "Chinese",
        "input": "I Love programming"
    }
)

AIMessage(content='我爱编程！ (Wǒ ài biānxué!)', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-16T18:34:23.4021336Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 13117679700, 'load_duration': 7305944800, 'prompt_eval_count': 29, 'prompt_eval_duration': 2932749000, 'eval_count': 17, 'eval_duration': 2804992000}, id='run-18ba6145-8cba-4e73-9086-e23512809be8-0', usage_metadata={'input_tokens': 29, 'output_tokens': 17, 'total_tokens': 46})

### Tool calling
we can use tool calling with an LLM that has been fine-tuned for tool use:
    ollama pull llama3-groq-tool-use
we can just pass normal Python function directly as tools

In [20]:
from typing import List

from langchain_ollama import ChatOllama
from typing_extensions import TypedDict

def validate_user(user_id: int, addresses: List) -> bool:
    """
        validate user using historical addresses.

        Args:
            user_id: (int) the user_id.
            addresses: previous addresses.
    """
    return True

llm = ChatOllama(
    # model="llama3-groq-tool-use",
    model="llama3.1",
    temperature=0,
).bind_tools([validate_user])

result = llm.invoke(
    "Could you validate user 123? They previously lived at"
    "123 Fake St in Boston MA and 234 Pretend Boulevard in"
    "Houston TX."
)
result.tool_calls

[{'name': 'validate_user',
  'args': {'addresses': '[{"street":"123 Fake St","city":"Boston","state":"MA"},{"street":"234 Pretend Boulevard","city":"Houston","state":"TX"}]',
   'user_id': 123},
  'id': 'ab3d27ce-9aef-46ff-ac77-04f064cd7f80',
  'type': 'tool_call'}]

In [15]:
from langchain_community.utilities import GoogleSerperAPIWrapper
import pprint
os.environ["SERPER_API_KEY"] = "b9bfe3246108bfd67d0673e18da06c973e1d84c1"
search = GoogleSerperAPIWrapper()
results = search.results("Apple Inc.")
pprint.pp(results)

{'searchParameters': {'q': 'Apple Inc.',
                      'gl': 'us',
                      'hl': 'en',
                      'type': 'search',
                      'num': 10,
                      'engine': 'google'},
 'knowledgeGraph': {'title': 'Apple',
                    'type': 'Technology company',
                    'website': 'http://www.apple.com/',
                    'imageUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQwGQRv5TjjkycpctY66mOg_e2-npacrmjAb6_jAWhzlzkFE3OTjxyzbA&s=0',
                    'description': 'Apple Inc. is an American multinational '
                                   'corporation and technology company '
                                   'headquartered in Cupertino, California, in '
                                   'Silicon Valley. It is best known for its '
                                   'consumer electronics, software, and '
                                   'services.',
                    'descriptionSource': 'Wikipe

In [20]:
from langchain_core.tools import tool, Tool
from langchain_community.utilities import GoogleSerperAPIWrapper

import os


# Set the Serper API key
os.environ["SERPER_API_KEY"] = "b9bfe3246108bfd67d0673e18da06c973e1d84c1"

@tool
def google_serper_search(query: str):
    """
    Google Serper 搜索引擎：输入一个字符串问题， 返回字符串结果
    """
    # Initialize the Google Serper API Wrapper
    search = GoogleSerperAPIWrapper()    

    # Initialize the language model
    # llm = ChatOllama(
    #     model="llama3.1",
    #     temperature=0,
    # )

    # # memory=SqliteSaver.from_conn_string(":memory:")

    # # Define the search tool
    # tools = [
    #     Tool(
    #         name="Intermediate Answer",
    #         func=search.run,
    #         description="Useful for when you need to ask with search",
    #     )
    # ]
    # self_ask_with_search = initialize_agent(
    #     tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
    # )

    results = search.results(query)
    print(results)
    return results

 

In [21]:
google_serper_search("巴黎奥运会奖牌信息？")

{'searchParameters': {'q': '巴黎奥运会奖牌信息？', 'gl': 'us', 'hl': 'en', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': '奖牌榜- 2024年巴黎奥运会 - Olympic Games', 'link': 'https://olympics.com/zh/paris-2024/medals', 'snippet': '2024年巴黎奥运会（7月26日至8月11日）官方奖牌榜。了解哪些运动员赢得奖牌、打破纪录。', 'date': 'Aug 11, 2024', 'attributes': {'Missing': '信息？ | Show results with:信息？'}, 'position': 1}, {'title': '2024年巴黎奥运会：中国代表团奖牌得主汇总 - Olympics', 'link': 'https://olympics.com/zh/news/paris-2024-team-china-medalists-overview', 'snippet': '截止至2024年8月11日，中国代表团在2024年巴黎奥运会上已经斩获了40金27银24铜合计91枚奖牌。 最后更新日期：2024年8月11日北京时间20:30.', 'date': 'Aug 11, 2024', 'position': 2}, {'title': '中国、英国、法国和澳大利亚将领跑2024 年巴黎奥运会奖牌榜| 尼尔森', 'link': 'https://www.nielsen.com/zh/news-center/2024/virtual-medal-table-forecast/', 'snippet': '获得奖牌最多的五个国家 (2024 年巴黎预测：112枚奖牌，2020 年东京：113 枚）。 美国有望再次在巴黎夏季奥运会虚拟奖牌榜上名列前茅。 目前Gracenote 预测的112 枚奖牌与 ...', 'position': 3}, {'title': '2024年巴黎奥运会跳水：比赛结果、获奖选手和奖牌榜一览 - Olympics', 'link': 'https://olympics.com/zh/

{'searchParameters': {'q': '巴黎奥运会奖牌信息？',
  'gl': 'us',
  'hl': 'en',
  'type': 'search',
  'num': 10,
  'engine': 'google'},
 'organic': [{'title': '奖牌榜- 2024年巴黎奥运会 - Olympic Games',
   'link': 'https://olympics.com/zh/paris-2024/medals',
   'snippet': '2024年巴黎奥运会（7月26日至8月11日）官方奖牌榜。了解哪些运动员赢得奖牌、打破纪录。',
   'date': 'Aug 11, 2024',
   'attributes': {'Missing': '信息？ | Show results with:信息？'},
   'position': 1},
  {'title': '2024年巴黎奥运会：中国代表团奖牌得主汇总 - Olympics',
   'link': 'https://olympics.com/zh/news/paris-2024-team-china-medalists-overview',
   'snippet': '截止至2024年8月11日，中国代表团在2024年巴黎奥运会上已经斩获了40金27银24铜合计91枚奖牌。 最后更新日期：2024年8月11日北京时间20:30.',
   'date': 'Aug 11, 2024',
   'position': 2},
  {'title': '中国、英国、法国和澳大利亚将领跑2024 年巴黎奥运会奖牌榜| 尼尔森',
   'link': 'https://www.nielsen.com/zh/news-center/2024/virtual-medal-table-forecast/',
   'snippet': '获得奖牌最多的五个国家 (2024 年巴黎预测：112枚奖牌，2020 年东京：113 枚）。 美国有望再次在巴黎夏季奥运会虚拟奖牌榜上名列前茅。 目前Gracenote 预测的112 枚奖牌与 ...',
   'position': 3},
  {'title': '2024年巴黎奥运会跳水：比赛结果、获奖选手

In [60]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver # an in-memory checkpointer
from langchain.agents import AgentType, initialize_agent
from langchain_ollama import ChatOllama  # Assuming this is the correct import for the language model

# Check if SqliteSaver is a subclass of BaseCheckpointSaver
assert issubclass(SqliteSaver, BaseCheckpointSaver), "SqliteSaver should be a subclass of BaseCheckpointSaver"

# Memorize in RAM
memory = MemorySaver()

# Initialize the language model
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
)

# Define the model and tools
model = llm
search = google_serper_search  # Ensure this is defined or imported
tools = [search]

# Create the agent executor
agent_executor = create_react_agent(model, tools, checkpointer=memory)
agent_executor = create_react_agent(model, tools)

# Use the agent
config = {"Configurable": {"thread_id": "1"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="How many people live in Canada?")]}, config):
    print(chunk)
    print("------")

{'agent': {'messages': [AIMessage(content='', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-20T15:23:19.2954549Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'google_serper_search', 'arguments': {'query': 'population of Canada'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 18954156600, 'load_duration': 17366200, 'prompt_eval_count': 163, 'prompt_eval_duration': 15345506000, 'eval_count': 21, 'eval_duration': 3589874000}, id='run-1bfac2f6-1c97-4817-a582-cb40950132ee-0', tool_calls=[{'name': 'google_serper_search', 'args': {'query': 'population of Canada'}, 'id': '3424594e-39e9-49b5-92bc-286f888c3087', 'type': 'tool_call'}], usage_metadata={'input_tokens': 163, 'output_tokens': 21, 'total_tokens': 184})]}}
------
{'searchParameters': {'q': 'population of Canada', 'gl': 'us', 'hl': 'en', 'type': 'search', 'num': 10, 'engine': 'google'}, 'answerBox': {'title': 'Canada / Population', 'answer': '38.93 million (2022)', 

In [59]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="How's weather today in Toronto")]}, config
):
    print(chunk)
    print("----------")

{'agent': {'messages': [AIMessage(content='', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-20T15:16:40.1755429Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'google_serper_search', 'arguments': {'query': 'Toronto weather today'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 18962070600, 'load_duration': 19200800, 'prompt_eval_count': 162, 'prompt_eval_duration': 15042799000, 'eval_count': 21, 'eval_duration': 3898395000}, id='run-d9b1aa3e-f7d1-45e0-bb92-0f0c7f17387b-0', tool_calls=[{'name': 'google_serper_search', 'args': {'query': 'Toronto weather today'}, 'id': 'c322d7ba-0f70-4f3b-a537-17903c31c54e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 162, 'output_tokens': 21, 'total_tokens': 183})]}}
----------
{'searchParameters': {'q': 'Toronto weather today', 'gl': 'us', 'hl': 'en', 'type': 'search', 'num': 10, 'engine': 'google'}, 'answerBox': {'title': 'Toronto, ON, Canada / Weather', 'answer': '62°F',